# Builder (빌더)

    "(생성자 오버라이딩 개념이 있는 프로그래밍 언어의 경우) 생성자에 전달할 인수가 복잡할 때 어쩌지?"

## 정의

파이썬에서는 특별한 경우가 아닌 한 빌더 패턴을 사용할 이유가 없다. 딕셔너리 언패킹을 통해 키워드 인자를 전달하는 방식, 함수 인자의 기본값을 할당하는 방식 등이 이미 존재하기 때문이다. 그러나 메서드 오버로딩을 지원하는 Java나 C#과 같은 프로그래밍 언어에서는 자주 사용되는 패턴이다.

해당 문제를 해결하기 위한 자바의 패턴은 다음과 같다.
- 점층적 생성자 : 여러 개의 생성자를 전부 정의하던가 (생성자 오버로딩)
- 자바 빈 : 객체를 생성하고 세터를 활용해 값을 설정하던가 (세터가 self를 return하여 체이닝 방식으로 구성, 불변한 객체를 설계하기가 불가능)
- 빌더 : Builder 생성자에 필수 인자, builder 객체에 여러 세터들을 두고, Builder의 메서드를 활용해 Product 객체를 생성하던가

그렇다면, 어떻게 사용될 수 있는가?
- 고전적 빌더 패턴 (파이썬에서 의미 없음)
- 객체의 생성 과정이 동일한 다른 여러 객체를 위해 일종의 '템플릿 메서드'처럼 빌더 패턴을 활용

그러나 두 번재 예제도 자주 사용될 것 같지는 않다.

## 구현

In [56]:
# 고전적 빌더 패턴의 구조
from __future__ import annotations

class Product:

    def __init__(self, builder: ProductBuilder):
        self.required_1 = builder.required_1
        self.required_2 = builder.required_2
        self.optional_1 = builder._optional_1
        self.optional_2 = builder._optional_2

class ProductBuilder:

    # 해당 Builder 객체를 Product 내부에 두어도 좋을 것 같다.
    # product = Product.Builder(...).optional...

    def __init__(self, required_1, required_2):
        self.required_1 = required_1
        self.required_2 = required_2
        self._optional_1 = None
        self._optional_2 = None

    def build(self):
        return Product(self)
    
    def optional_1(self, optional_1):
        self._optional_1=optional_1
        return self

    def optional_2(self, optional_2):
        self._optional_2=optional_2
        return self

product_1 = ProductBuilder(1, 2).build()
product_2 = ProductBuilder(1, 2).optional_1(3).optional_2(4).build()

print(product_1.__dict__)
print(product_2.__dict__)

{'required_1': 1, 'required_2': 2, 'optional_1': None, 'optional_2': None}
{'required_1': 1, 'required_2': 2, 'optional_1': 3, 'optional_2': 4}


In [53]:
# 하나의 생성 과정을 여러 타입의 객체 생성에 활용하는 패턴
from abc import ABCMeta, abstractmethod
from typing import Any


# 하나의 생성 과정...
def build(builder: AbstractBuilder):
    builder.part_1(1)
    return builder.build()


# Products...
class Some:

    def __init__(self, builder: SomeBuilder):
        ...

class Tome:

    def __init__(self, builder: TomeBuilder):
        ...


# Builders...
class AbstractBuilder(metaclass=ABCMeta):

    @abstractmethod
    def part_1(self, value):
        ...
    
    @abstractmethod
    def build(self) -> Any:
        ...


class SomeBuilder(AbstractBuilder):

    def __init__(self):
        self._part_1 = None

    def part_1(self, value):
        self._part_1 = value
    
    def build(self):
        return Some(self)


class TomeBuilder(AbstractBuilder):

    def __init__(self):
        self._part_1 = None

    def part_1(self, value):
        self._part_1 = value
    
    def build(self):
        return Tome(self)


if __name__ == "__main__":
    some_builder = SomeBuilder()
    some = build(some_builder)

    tome_builder = TomeBuilder()
    tome = build(tome_builder)

    print(type(some))
    print(type(tome))

<class '__main__.Some'>
<class '__main__.Tome'>
